In [ ]:
from nbdev import *
%nbdev_default_export core

Cells will be exported to RemoteConnect.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# core

> This is core functionality for RemoteConnect

In [ ]:
%nbdev_hide
from nbdev.showdoc import *

In [ ]:
%nbdev_export_and_show
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
import os
import subprocess
from pyngrok import ngrok

In [ ]:
%nbdev_export_and_show
EXTENSIONS = ["ms-python.python", "jithurjacob.nbpreviewer"]

In [ ]:
%nbdev_export
def say_hello(to):
    print(f"Say hello to {to}")

In [ ]:
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    print(f"Disconnecting {public_url}")
    ngrok.disconnect(public_url)
    
url = ngrok.connect(port=10000, options={"bind_tls":True})
print(f"Code server can be assesed on : {url}")

Disconnecting https://8aa1ab1b87eb.ngrok.io
Code server can be assesed on : https://7c31f6001eeb.ngrok.io


In [ ]:
%nbdev_export
def connect_to_ngrok(port):
    active_tunnels = ngrok.get_tunnels()
    for tunnel in active_tunnels:
        public_url = tunnel.public_url
        print(f"Disconnecting {public_url}")
        ngrok.disconnect(public_url)
    url = ngrok.connect(port=port, options={"bind_tls":True})
    print(f"Code server can be assesed on : {url}")
    return url 

In [ ]:
%nbdev_export_and_show
class RemoteCode:
    """ Install and launch an instance of Remote Code"""
    def __init__(self, port=10000, password=None):
        self.port = port
        self.password = password
        self._mount = IN_COLAB
        self.url = None
        self._install_code()
        self._install_extensions()
        self._start_server()
        self._run_code()
        
    def _install_code(self):
        subprocess.run(
            ["wget", "https://code-server.dev/install.sh"], stdout=subprocess.PIPE
        )
        subprocess.run(["sh", "install.sh"], stdout=subprocess.PIPE)

    def _install_extensions(self):
        for ext in EXTENSIONS:
            subprocess.run(["code-server", "--install-extension", f"{ext}"]) 
            
    def _start_server(self):
        self.url = connect_to_ngrok(self.port)
        
    
    def _run_code(self):
        os.system(f"fuser -n tcp -k {self.port}")
        if IN_COLAB:
            drive.mount("/content/drive")
        if self.password:
            code_cmd = f"PASSWORD={self.password} code-server --port {self.port} --disable-telemetry"
        else:
            code_cmd = f"code-server --port {self.port} --auth none --disable-telemetry"
        with subprocess.Popen(
            [code_cmd],
            shell=True,
            stdout=subprocess.PIPE,
            bufsize=1,
            universal_newlines=True,
        ) as proc:
            for line in proc.stdout:
                print(line, end="")
        
    def __str__(self):
        return f"Remote Code runnning on {self.url}:{self.port} with password:{self.password} in colab envrionment:{IN_COLAB}"
        

In [ ]:
#print(RemoteCode())

In [ ]:
active_tunnels, url

([<NgrokTunnel: "https://8aa1ab1b87eb.ngrok.io" -> "http://localhost:10000">],
 'https://7c31f6001eeb.ngrok.io')

In [ ]:
# os.system(f"fuser -n tcp -k {111111111}")